In [2]:
# 类别特征和连续性特征 一阶交叉

def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max', '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
            })
            df = df.merge(feat, on=f1, how='left')
    return (df)

data = cross_cat_num(data, num_col, cat_col)  # 一阶交叉
print('一阶交叉特征处理后：', data.shape)

In [ ]:
# count 编码
def count_coding(df, fea_col):
    for f in fea_col:
        df[f'{f}_count'] = df[f].map(df[f].value_counts())
    return (df)

data = count_coding(data,col_cat)


# 类别特征二阶交叉
f_pairs = [['是否双频','信用等级代码'],['婚姻状况','预计收入'],['婚姻状况','信用等级代码'],['家庭成人人数','家庭中唯一订阅者的数量'],['信息库匹配','账户消费限额'],
           ['信用等级代码','信息库匹配'],['地理区域','家庭活跃用户数'],['家庭活跃用户数','信用等级代码'],['是否翻新机','家庭中唯一订阅者的数量']]

def cross_qua_cat_num(df):
    for f_pair in tqdm(f_pairs):
        ### 共现次数
        df['_'.join(f_pair) + '_count'] = df.groupby(f_pair)['客户ID'].transform('count')
        ### n unique、熵
        df = df.merge(df.groupby(f_pair[0], as_index=False)[f_pair[1]].agg({
            '{}_{}_nunique'.format(f_pair[0], f_pair[1]): 'nunique',
            '{}_{}_ent'.format(f_pair[0], f_pair[1]): lambda x: entropy(x.value_counts() / x.shape[0])
        }), on=f_pair[0], how='left')
        df = df.merge(df.groupby(f_pair[1], as_index=False)[f_pair[0]].agg({
            '{}_{}_nunique'.format(f_pair[1], f_pair[0]): 'nunique',
            '{}_{}_ent'.format(f_pair[1], f_pair[0]): lambda x: entropy(x.value_counts() / x.shape[0])
        }), on=f_pair[1], how='left')
        ### 比例偏好
        df['{}_in_{}_prop'.format(f_pair[0], f_pair[1])] = df['_'.join(f_pair) + '_count'] / df[f_pair[1] + '_count']
        df['{}_in_{}_prop'.format(f_pair[1], f_pair[0])] = df['_'.join(f_pair) + '_count'] / df[f_pair[0] + '_count']
    return (df)

cross_qua_cat_num(data)